# Implementation of baseline using Artificial Neural Network with Original + Win-loss + feature vectors feature set and SA

Necessary files: logistic_regression_functions.py, ann_functions.py, testing_functions.py, win_loss_functions.py, feature_vectors_functions, SA_17.csv, SA_18.csv, SA_19.csv, A_SA.csv, H_SA.csv, A_SA_before_threshold.csv, H_SA_before_threshold.csv

In [ ]:
import logistic_regression_functions
import ann_functions
import testing_functions
import win_loss_functions
import feature_vectors_functions
import pandas as pd
from datetime import datetime
import requests
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tqdm import tqdm

The baseline is implemented according to the paper An Improved Prediction System for Football a Match Result bu C. P. Igiri and E. O. Nwachukwu from 2014. The original features as described in the paper are used and the model is evaluated on the SA dataset.

The authors used only one season for training, so I did the same. I used seasons 2016/17 for training season 2017/18 as validation set and the second half of season 2018/19 for testing. The validation set is used to select the best checkpoint of the training according to the validation accuracy.

In [ ]:
X_train_win_loss, y_train = win_loss_functions.create_data(['SA_17.csv'], skip_rounds = 6, return_names=True)
results_val_win_loss, matches_per_round = win_loss_functions.create_data_single('SA_18.csv', ['SA_17.csv'], skip_rounds = 6, return_names=True)
# Dates are returned as well for dividing testing season into slices
results_test_win_loss, matches_per_round = win_loss_functions.create_data_single('SA_19.csv', ['SA_17.csv', 'SA_18.csv'],
                                return_dates=True, skip_rounds = 6, return_names=True)

Processing SA_17.csv season file.


In [ ]:
results_train_originals, matches_per_round = ann_functions.create_data_single('SA_17.csv', 31, 17,
                                   logistic_regression_functions.team_names_map_sa,
                                   logistic_regression_functions.secondary_team_names_map_sa)
results_val_originals, matches_per_round = ann_functions.create_data_single('SA_18.csv', 31, 18,
                                  logistic_regression_functions.team_names_map_sa,
                                  logistic_regression_functions.secondary_team_names_map_sa)
# Dates are returned as well for dividing testing season into slices
results_test_originals, matches_per_round = ann_functions.create_data_single('SA_19.csv', 31, 19,
                                  logistic_regression_functions.team_names_map_sa,
                                  logistic_regression_functions.secondary_team_names_map_sa,
                                  return_dates=True)
X_train_originals = results_train_originals.drop('FTR', axis=1)
y_train_originals = results_train_originals['FTR']

In [ ]:
# Concatenating the two feature sets together
X_train_mix = pd.concat([X_train_originals, X_train_win_loss], axis=1)
results_val_mix = pd.concat([results_val_originals, results_val_win_loss], axis=1)
results_test_mix = pd.concat([results_test_originals, results_test_win_loss], axis=1)
X_train_mix = X_train_mix.loc[:,~X_train_mix.columns.duplicated()].copy()
results_val_mix = results_val_mix.loc[:,~results_val_mix.columns.duplicated()].copy()
results_test_mix = results_test_mix.loc[:,~results_test_mix.columns.duplicated()].copy()

In [ ]:
A = pd.read_csv('A_SA.csv')
H = pd.read_csv('H_SA.csv')
A_before_threshold = pd.read_csv('A_SA_before_threshold.csv')
H_before_threshold = pd.read_csv('H_SA_before_threshold.csv')

In [ ]:
# Adding the feature vectors to the features
X_train = feature_vectors_functions.add_feature_vector(X_train_mix, A, H)
results_val = feature_vectors_functions.add_feature_vector(results_val_mix, A, H)
results_test = feature_vectors_functions.add_feature_vector(results_test_mix, A, H)

In [ ]:
X_val = results_val.drop('FTR', axis=1)
y_val = results_val['FTR']
# The classes need to go from 0 to 2 not from -1 to 1.
y_train += 1
y_val += 1

They didnt say what ANN architecture they used. I used 3 dense layers with 64, 32 and 16 neurons. I used dropout to overcome overfitting.

I always let the model train for 100 epochs and took the weights from the best epoch according to validation accuracy.

To use all the data available, the first half of the testing season was added to the training data.

In [ ]:
# Some rounds in the beginning are ignored, this is the correct index
# of the start of the second half of the season
start_test_index = 13 * matches_per_round

In [ ]:
X_test_to_append, y_test_to_append = testing_functions.prepare_test_to_append(results_test,
                                                                              start_test_index)
y_test_to_append += 1

In [ ]:
# Adding 1st half of testing season to the training data
X_train = pd.concat([X_train, X_test_to_append])
y_train = pd.concat([y_train, y_test_to_append])
# Rounds of the testing dataset
slices = testing_functions.get_slices(results_test, matches_per_round,
                                      start_test_index)
weighted_sum = 0
sum = 0
for slc in tqdm(slices):
  # Creating the model
  model = ann_functions.func_model((X_train.shape[1],))
  model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
  mc = tf.keras.callbacks.ModelCheckpoint('./weights_model.h5',
                                     monitor='val_accuracy',
                                     save_weights_only=True,
                                     save_best_only=True)
  X_test = slc.drop(['FTR', 'Date'], axis=1)
  y_test = slc['FTR']
  y_test += 1
  # Train the model
  history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=8,
                    validation_data=(X_val, y_val),
                    verbose = 0,
                    callbacks=[mc])
  # Load the best checkpoint
  model.load_weights('weights_model.h5')
  weighted_sum += (model.evaluate(X_test, y_test)[1] * len(y_test))
  sum += len(y_test)
  # Add the round to the training dataset
  X_train = pd.concat([X_train, X_test])
  y_train = pd.concat([y_train, y_test])
print('')
print(weighted_sum / sum)

  0%|          | 0/15 [00:00<?, ?it/s]

1/1 [==============================] - 0s 155ms/step - loss: 1.0065 - accuracy: 0.5385


  7%|▋         | 1/15 [00:21<04:58, 21.35s/it]

1/1 [==============================] - 0s 137ms/step - loss: 1.4098 - accuracy: 0.2667


 13%|█▎        | 2/15 [00:42<04:37, 21.38s/it]

...

1/1 [==============================] - 0s 141ms/step - loss: 0.9479 - accuracy: 0.5000


 93%|█████████▎| 14/15 [05:34<00:31, 31.79s/it]

1/1 [==============================] - 0s 141ms/step - loss: 1.1815 - accuracy: 0.4545


100%|██████████| 15/15 [05:57<00:00, 23.81s/it]


0.5210526380099748


The testing accuracy is 52.11%.

Experimenting with feature vectors extracted before applying the threshold.

In [ ]:
# y_train was changed, load it again
X_train_win_loss, y_train = win_loss_functions.create_data(['SA_17.csv'], skip_rounds = 6, return_names=True)

Processing SA_17.csv season file.


In [ ]:
# Adding the feature vectors to the features
X_train = feature_vectors_functions.add_feature_vector(X_train_mix, A_before_threshold, H_before_threshold)
results_val = feature_vectors_functions.add_feature_vector(results_val_mix, A_before_threshold, H_before_threshold)
results_test = feature_vectors_functions.add_feature_vector(results_test_mix, A_before_threshold, H_before_threshold)

In [ ]:
X_val = results_val.drop('FTR', axis=1)
y_val = results_val['FTR']
# The classes need to go from 0 to 2 not from -1 to 1.
y_train += 1
y_val += 1

In [ ]:
X_test_to_append, y_test_to_append = testing_functions.prepare_test_to_append(results_test,
                                                                              start_test_index)
y_test_to_append += 1

In [ ]:
# Adding 1st half of testing season to the training data
X_train = pd.concat([X_train, X_test_to_append])
y_train = pd.concat([y_train, y_test_to_append])
# Rounds of the testing dataset
slices = testing_functions.get_slices(results_test, matches_per_round,
                                      start_test_index)
weighted_sum = 0
sum = 0
for slc in tqdm(slices):
  # Creating the model
  model = ann_functions.func_model((X_train.shape[1],))
  model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
  mc = tf.keras.callbacks.ModelCheckpoint('./weights_model.h5',
                                     monitor='val_accuracy',
                                     save_weights_only=True,
                                     save_best_only=True)
  X_test = slc.drop(['FTR', 'Date'], axis=1)
  y_test = slc['FTR']
  y_test += 1
  # Train the model
  history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=8,
                    validation_data=(X_val, y_val),
                    verbose = 0,
                    callbacks=[mc])
  # Load the best checkpoint
  model.load_weights('weights_model.h5')
  weighted_sum += (model.evaluate(X_test, y_test)[1] * len(y_test))
  sum += len(y_test)
  # Add the round to the training dataset
  X_train = pd.concat([X_train, X_test])
  y_train = pd.concat([y_train, y_test])
print('')
print(weighted_sum / sum)

  0%|          | 0/15 [00:00<?, ?it/s]

1/1 [==============================] - 0s 117ms/step - loss: 0.9851 - accuracy: 0.5385


  7%|▋         | 1/15 [00:16<03:49, 16.40s/it]

1/1 [==============================] - 0s 138ms/step - loss: 1.5508 - accuracy: 0.2000


 13%|█▎        | 2/15 [00:32<03:28, 16.04s/it]

...

1/1 [==============================] - 0s 119ms/step - loss: 1.0674 - accuracy: 0.5000


 93%|█████████▎| 14/15 [04:26<00:20, 20.61s/it]

1/1 [==============================] - 0s 120ms/step - loss: 1.0885 - accuracy: 0.3636


100%|██████████| 15/15 [04:45<00:00, 19.01s/it]


0.5210526384021107


The testing accuracy is 52.11%.

Experimenting with feature vectors including home feature vectors for the away teams and away feature vectors for the home teams.

In [ ]:
# y_train was changed, load it again
X_train_win_loss, y_train = win_loss_functions.create_data(['SA_17.csv'], skip_rounds = 6, return_names=True)

Processing SA_17.csv season file.


In [ ]:
# Adding the feature vectors to the features
X_train = feature_vectors_functions.add_feature_vector(X_train_mix, A, H, include_all=True)
results_val = feature_vectors_functions.add_feature_vector(results_val_mix, A, H, include_all=True)
results_test = feature_vectors_functions.add_feature_vector(results_test_mix, A, H, include_all=True)

In [ ]:
X_val = results_val.drop('FTR', axis=1)
y_val = results_val['FTR']
# The classes need to go from 0 to 2 not from -1 to 1.
y_train += 1
y_val += 1

In [ ]:
X_test_to_append, y_test_to_append = testing_functions.prepare_test_to_append(results_test,
                                                                              start_test_index)
y_test_to_append += 1

In [ ]:
# Adding 1st half of testing season to the training data
X_train = pd.concat([X_train, X_test_to_append])
y_train = pd.concat([y_train, y_test_to_append])
# Rounds of the testing dataset
slices = testing_functions.get_slices(results_test, matches_per_round,
                                      start_test_index)
weighted_sum = 0
sum = 0
for slc in tqdm(slices):
  # Creating the model
  model = ann_functions.func_model((X_train.shape[1],))
  model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
  mc = tf.keras.callbacks.ModelCheckpoint('./weights_model.h5',
                                     monitor='val_accuracy',
                                     save_weights_only=True,
                                     save_best_only=True)
  X_test = slc.drop(['FTR', 'Date'], axis=1)
  y_test = slc['FTR']
  y_test += 1
  # Train the model
  history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=8,
                    validation_data=(X_val, y_val),
                    verbose = 0,
                    callbacks=[mc])
  # Load the best checkpoint
  model.load_weights('weights_model.h5')
  weighted_sum += (model.evaluate(X_test, y_test)[1] * len(y_test))
  sum += len(y_test)
  # Add the round to the training dataset
  X_train = pd.concat([X_train, X_test])
  y_train = pd.concat([y_train, y_test])
print('')
print(weighted_sum / sum)

  0%|          | 0/15 [00:00<?, ?it/s]

1/1 [==============================] - 0s 129ms/step - loss: 0.8646 - accuracy: 0.5385


  7%|▋         | 1/15 [00:17<04:03, 17.36s/it]

1/1 [==============================] - 0s 130ms/step - loss: 1.2648 - accuracy: 0.3333


 13%|█▎        | 2/15 [00:35<03:52, 17.88s/it]

...

1/1 [==============================] - 0s 147ms/step - loss: 1.0149 - accuracy: 0.5000


 93%|█████████▎| 14/15 [04:41<00:21, 21.30s/it]

1/1 [==============================] - 0s 140ms/step - loss: 1.0289 - accuracy: 0.4545


100%|██████████| 15/15 [05:02<00:00, 20.18s/it]


0.5263157952772943


The testing accuracy is 52.63%.

Experimenting with feature vectors extracted before applying the threshold and including home feature vectors for the away teams and away feature vectors for the home teams.

In [ ]:
# y_train was changed, load it again
X_train_win_loss, y_train = win_loss_functions.create_data(['SA_17.csv'], skip_rounds = 6, return_names=True)

Processing SA_17.csv season file.


In [ ]:
# Adding the feature vectors to the features
X_train = feature_vectors_functions.add_feature_vector(X_train_mix, A_before_threshold, H, include_all=True)
results_val = feature_vectors_functions.add_feature_vector(results_val_mix, A_before_threshold, H_before_threshold, include_all=True)
results_test = feature_vectors_functions.add_feature_vector(results_test_mix, A_before_threshold, H_before_threshold, include_all=True)

In [ ]:
X_val = results_val.drop('FTR', axis=1)
y_val = results_val['FTR']
# The classes need to go from 0 to 2 not from -1 to 1.
y_train += 1
y_val += 1

In [ ]:
X_test_to_append, y_test_to_append = testing_functions.prepare_test_to_append(results_test,
                                                                              start_test_index)
y_test_to_append += 1

In [ ]:
# Adding 1st half of testing season to the training data
X_train = pd.concat([X_train, X_test_to_append])
y_train = pd.concat([y_train, y_test_to_append])
# Rounds of the testing dataset
slices = testing_functions.get_slices(results_test, matches_per_round,
                                      start_test_index)
weighted_sum = 0
sum = 0
for slc in tqdm(slices):
  # Creating the model
  model = ann_functions.func_model((X_train.shape[1],))
  model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
  mc = tf.keras.callbacks.ModelCheckpoint('./weights_model.h5',
                                     monitor='val_accuracy',
                                     save_weights_only=True,
                                     save_best_only=True)
  X_test = slc.drop(['FTR', 'Date'], axis=1)
  y_test = slc['FTR']
  y_test += 1
  # Train the model
  history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=8,
                    validation_data=(X_val, y_val),
                    verbose = 0,
                    callbacks=[mc])
  # Load the best checkpoint
  model.load_weights('weights_model.h5')
  weighted_sum += (model.evaluate(X_test, y_test)[1] * len(y_test))
  sum += len(y_test)
  # Add the round to the training dataset
  X_train = pd.concat([X_train, X_test])
  y_train = pd.concat([y_train, y_test])
print('')
print(weighted_sum / sum)

  0%|          | 0/15 [00:00<?, ?it/s]

1/1 [==============================] - 0s 144ms/step - loss: 0.7752 - accuracy: 0.6923


  7%|▋         | 1/15 [00:17<04:03, 17.42s/it]

1/1 [==============================] - 0s 124ms/step - loss: 2.0590 - accuracy: 0.2000


 13%|█▎        | 2/15 [00:38<04:16, 19.70s/it]

...

1/1 [==============================] - 0s 128ms/step - loss: 0.8960 - accuracy: 0.5625


 93%|█████████▎| 14/15 [04:24<00:20, 20.17s/it]

1/1 [==============================] - 0s 125ms/step - loss: 1.0238 - accuracy: 0.4545


100%|██████████| 15/15 [04:45<00:00, 19.05s/it]


0.5263157986496624


The testing accuracy is 52.63%.